# Introducción

En este listado de ejercicios propuestos podrás hacerte una idea del tipo de consultas que se pueden pedir en la prueba.

Recuerda consultar la referencia de los Models de Odoo para poder resolverlos. Una vez tengas el modo Desarrollador activado en los Ajustes, la referencia se encuentra en el menú **Técnico** → **Modelos** (dentro del apartado **Estructura de la base de datos**).

Asegúrate de contar con los datos de prueba en tu sistema.

Los resultados que aparecen debajo de los bloques de código son orientativos, para que puedas hacerte una idea de lo que debes obtener. Pueden variar dependiendo de la información que tengas en tu base de datos.

# Conexión con la API

Recuerda que para poder realizar la conexión, necesitamos los siguientes parámetros de nuestra instancia de Odoo:


* **URL**: Por ejemplo, `https://edu-test.odoo.com`.
*   **Nombre de la BD**: Por ejemplo, `edu-test`.
* **Nombre de usuario**: El correo electrónico con el que iniciamos sesión en nuestra instancia. Por ejemplo, `abcdefg123@g.educaand.es`.
* **Contraseña del usuario**: Por ejemplo, `password`.

**Importante**: Cuando damos de alta una instancia dominio.odoo.com, la contraseña que tenemos es de la cuenta en odoo.com, pero no se genera automáticamente una contraseña para el usuario administrador. Por lo tanto, tenemos que generar una (aconsejo que sea la misma, para evitar confusiones posteriores). Para ello, seguimos los siguientes pasos:


1.   Iniciamos sesión en nuestra instancia de Odoo.
2.   Pulsamos en la aplicación `Ajustes`.
3. En el apartado `OPCIONES GENERALES` → `Usuarios`, pulsamos en `Administrar usuarios`.
4. Hacemos clic en nuestro usuario.
5. Pulsamos en `Acción` → `Cambiar la contraseña`.
6. Establecemos la contraseña que queramos.

Una vez lo hemos hecho, ya podemos almacenar en variables los datos necesarios para la conexión.

Modifica el siguiente bloque de código con los datos de tu instancia y ejecutálo.




In [ ]:
url = 'https://edu-test.odoo.com'
db = 'edu-test'
username = 'abcdefg123@g.educaand.es'
password = 'password'

Ejecuta ahora el siguiente para poder comenzar a realizar las peticiones con el uid:

In [ ]:
import xmlrpc.client
common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))

# 1. Cuenta las ventas mayores de 1000

Muestra cuántas ventas se han realizado con una cantidad total mayor que 1000.

In [ ]:
models.execute_kw(db, uid, password,
    'sale.order', 'search_count',
    [[['amount_total', '>', '1000']]])

14

# 2. Dos ventas mayores de 1000

Muestra los identificadores que ocupan el lugar 3º y 4º de las ventas con una cantidad total mayor que 1000 e indica cuáles han sido esas cantidades.





In [ ]:
models.execute_kw(db, uid, password,
    'sale.order', 'search_read',
    [[['amount_total', '>', '1000']]],
    {'fields': ['amount_total'], 'offset': 2, 'limit': 2})

[{'amount_total': 1475.0, 'id': 22}, {'amount_total': 2175.0, 'id': 21}]

Calcula la suma de ambas cantidades:

In [ ]:
ventas = models.execute_kw(db, uid, password,
    'sale.order', 'search_read',
    [[['amount_total', '>', '1000']]],
    {'fields': ['amount_total'], 'offset': 2, 'limit': 2})
print(ventas[0]['amount_total']+ventas[1]['amount_total'])

3650.0


# 3. Escasez de productos

Muestra el nombre de los productos de los que haya un stock menor de 15 y mayor o igual que 1.

In [ ]:
productos = models.execute_kw(db, uid, password,
    'product.product', 'search_read',
    [[['qty_available', '<', 15],['qty_available', '>=', 1]]],
    {'fields': ['name']})

for producto in productos:
  print(producto['name'])

Office Chair Black
Corner Desk Left Sit


# 4. Contactos estadounidenses

Muestra los correos electrónicos de los contactos estadounidenses.

In [ ]:
contactos = models.execute_kw(db, uid, password,
    'res.partner', 'search_read', 
    [[['country_id', '=', 233]]],
    {'fields': ['email']})

for contacto in contactos:
  print(contacto['email'])

vauxoo@yourcompany.example.com
brandon.freeman55@example.com
colleen.diaz83@example.com
nicole.ford75@example.com
info@agrolait.com
addison.olson28@example.com
douglas.fletcher51@example.com
floyd.steward34@example.com
john.b@tech.info
edwin.hansen58@example.com
jesse.brown74@example.com
oscar.morgan11@example.com
soham.palmer15@example.com
lumber-inv92@example.com
lorraine.douglas35@example.com
chicago@yourcompany.com
jeff.lawson52@example.com
info@yourcompany.com
chester.reed79@example.com
dwayne.newman28@example.com
info@deltapc.com
billy.fox45@example.com
edith.sanchez68@example.com
julie.richards84@example.com
kim.snyder96@example.com
sandra.neal80@example.com
theodore.gardner36@example.com
travis.mendoza24@example.com
jackson.group82@example.com
gordon.owens47@example.com
toni.rhodes11@example.com
bhu.a100@ic.example.com
ron.gibson76@example.com
tom.ruiz89@example.com
willie.burke80@example.com
admin@yourcompany.example.com


# 5. Llama a los despistados

Muestra los números de teléfono de aquellos contactos que tienen algún mensaje sin leer.

In [ ]:
contactos = models.execute_kw(db, uid, password,
    'res.partner', 'search_read', 
    [[['message_unread', '=', True]]],
    {'fields': ['phone','message_unread']})

for contacto in contactos:
  # Por alguna razón desconocida el filtro de search no funciona, pero puede hacerse con un condicional fácilmente
  if contacto['message_unread']:
    print(contacto['phone'])

+1 (650) 555-0111 


# 6. Brecha de seguridad

Ha habido una fuga de datos y parece que algunas cuentas de los usuarios han sido comprometidas. Muestra el nombre, el correo electrónico y el teléfono de aquellos que no tengan configurado el doble factor de autenticación para avisarles inmediatamente.

In [ ]:
models.execute_kw(db, uid, password,
    'res.users', 'search_read', 
    [[['totp_enabled', '=', False]]],
    {'fields': ['name','email','phone']})

[{'email': 'admin@yourcompany.example.com',
  'id': 2,
  'name': 'Mitchell Admin',
  'phone': '+1 555-555-5555'}]

# 7. Agregando productos

La empresa va a comenzar a vender consolas. Crea los productos con sus respectivos precios usando la API y almacena los identificadores en una lista llamada `ids`. Los productos son:
- Nombre: Playstation 5. Precio 499€
- Nombre: Nintendo Switch. Precio 299€
- Nombre: Xbox Series S. Precio 299€
- Nombre: Xbox Series X. Precio 499€
- Nombre: Playstation 4. Precio 299€

In [ ]:
products = [['Playstation 5',499],['Nintendo Switch',299],['Xbox Series S',299],['Xbox Series X',499],['Playstation 4',299]]
ids = []

for name,price in products:  
  id = models.execute_kw(db, uid, password, 'product.product', 'create', [{
    'name': name, 'price': price,
    }])
  ids.append(id)

print(ids)

[47, 48, 49, 50, 51]


# 8. Actualizando productos

Actualiza los productos añadidos en el ejercicio anterior, aumentando su precio en 100, ya que comienza a haber escasez de los mismos.

In [ ]:
models.execute_kw(db, uid, password, 'product.product', 'write', [[ids[0],ids[3]], {
    'price': 599
}])
models.execute_kw(db, uid, password, 'product.product', 'write', [[ids[1],ids[2],ids[4]], {
    'price': 499
}])

True

# 9. Elimina los productos

A un empleado de la compañía se le ha olvidado declarar que ibáis a comenzar a vender consolas. Mientras se soluciona el problema legal, es mejor eliminar esos productos del inventario.

In [ ]:
models.execute_kw(db, uid, password, 'product.product', 'unlink', [ids])

True

# 10. Mi primer script: Un buscador de productos por precio

Elabora un script en Visual Studio Code que realice lo siguiente:
1. Pregunte al usuario por el precio que desea buscar.
2. Pregunte al usuario si desea mostrar los productos que tengan un precio igual, superior o menor a ese precio de referencia.
3. Muestre al usuario los nombres de los productos que cumplen esa condición.

A continuación, copia y pega ese script en el siguiente bloque de código:

In [ ]:
url = 'https://edu-test.odoo.com'
db = 'edu-test'
username = 'abcdefg123@g.educaand.es'
password = 'password'

import xmlrpc.client
common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))

precio = float(input("¿Cuál es el precio que desea buscar?: "))

comparacion = input("Indica si desea localizar los productos que tengan un precio mayor, menor o igual: ")

if comparacion == 'mayor':
    
    productos = models.execute_kw(db, uid, password,
    'product.product', 'search_read',
    [[['list_price', '>', precio]]],
    {'fields': ['name']})
    if len(productos)>0:
        print("Los productos con un precio mayor que "+str(precio)+" son:")
        for producto in productos:
            print(producto['name'])
    else:
        print("No hay ningún producto con un precio mayor que ese")
elif comparacion == 'menor':
    productos = models.execute_kw(db, uid, password,
    'product.product', 'search_read',
    [[['list_price', '<', precio]]],
    {'fields': ['name']})
    if len(productos)>0:
        print("Los productos con un precio menor que "+str(precio)+" son:")
        for producto in productos:
            print(producto['name'])
    else:
        print("No hay ningún producto con un precio menor que ese")
else:
    productos = models.execute_kw(db, uid, password,
    'product.product', 'search_read',
    [[['list_price', '=', precio]]],
    {'fields': ['name']})
    if len(productos)>0:
        print("Los productos con el precio "+str(precio)+" son:")
        for producto in productos:
            print(producto['name'])
    else:
        print("No hay ningún producto con ese precio")

# 11. Mi segundo script: El límite es tu imaginación

Elabora otro script usando la API de Odoo que realice lo que quieras. Cuando más complejo sea el reto que te propongas, más aprenderás.

Escribe a continuación la descripción de tu script y copia/pega el código en el bloque contiguo:

